#**Launch Sites Locations Analysis with Folium**

In [ ]:
!pip3 install folium
!pip3 install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=049613263648c8779be0a5012840f9ac1778b2e7d7aae52a35ad70ecdb2887c4
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
import folium
import wget
import pandas as pd

In [ ]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

##Task 1: Mark all launch sites on a map

In [ ]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

In [ ]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
# .first(): Bu kısım, her grubun ilk satırını seçer.
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [ ]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [ ]:
# colors = ["#7f7f7f", "#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf"]

# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#2ca02c', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#1f77b4;"><b>%s</b></div>' % 'NASA JSC', #d35400
        )
    )


site_map.add_child(circle)
site_map.add_child(marker)

In [ ]:
# Initial the map
# site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

# For each launch site, add a CircleMarker object based on its coordinate (Lat, Long) values.
# In addition, add Launch site name as a popup label

def add_site_marker(coordinate, launch_site_name):
    circle_marker = folium.CircleMarker(
        location=coordinate,
        radius=10,  # Adjusted for visibility
        color='#d35400',
        fill=True,
        fill_color='#d35400'
    )
    marker = folium.Marker(
        location=coordinate,
        icon=folium.DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % launch_site_name,
        )
    )
    circle_marker.add_to(site_map)
    marker.add_to(site_map)

# Assuming launch_sites_df is a DataFrame with columns 'Lat', 'Long', 'Launch Site'
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    launch_site_name = row['Launch Site']

    add_site_marker(coordinate, launch_site_name)

# Display the map
site_map

##Task 2: Mark the success/failed launches for each site on the map

In [ ]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [ ]:
marker_cluster = MarkerCluster()

In [ ]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [ ]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed,
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color'])

for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    coordinate = [record['Lat'], record['Long']]
    marker = folium.Marker(coordinate, icon=folium.Icon(color='white', icon_color=record['marker_color']))
    marker_cluster.add_child(marker)

site_map

##TASK 3: Calculate the distances between a launch site to its proximities

In [ ]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map, (shown on the top-right)
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [ ]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [ ]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163

# Given coordinates
launch_site_lat = 28.56367
launch_site_lon = -80.57163
coastline_lat = 28.56342  # provide the latitude of the coastline
coastline_lon = -80.56744  # provide the longitude of the coastline

distance = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

print(f"The distance to the closest coastline is {distance} kilometers.")

The distance to the closest coastline is 0.410271918104809 kilometers.


In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

launch_coor = [launch_site_lat, launch_site_lon]
coastline_coor = [coastline_lat, coastline_lon]

# Create a map centered on the launch site
map_ = folium.Map(location=launch_coor, zoom_start=15)

# Create a marker for the launch site
folium.Marker(
    location=launch_coor,
    icon=folium.Icon(color='red', icon='rocket', prefix='fa')
).add_to(map_)

distance_marker = folium.Marker(coastline_coor, icon=DivIcon(
  icon_size=(20,20),
  icon_anchor=(0,0),
  html='<div style="font-size:12; color:#d35400;"><b>%s</b></div> '% "{:10.2f} KM". format(distance)
))
distance_marker.add_to(map_)
map_

In [ ]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
# site_map.add_child(lines)


# Given coordinates
launch_site_lat = 28.56341
launch_site_lon = -80.57678
railway_lat = 28.57205
railway_lon = -80.58527

# Calculating distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

# Create a map centered on the launch site
m = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Create a PolyLine object with the coastline and launch site coordinates
lines = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [railway_lat, railway_lon]],
    weight=1
)

# Add the PolyLine to the map
lines.add_to(m)

# Create a marker for the coastline point and add the distance
distance_marker = folium.Marker(
    location=[railway_lat, railway_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)
distance_marker.add_to(m)


marker_cluster = MarkerCluster()

# Create a function to assign marker colors based on the value of the class column
def assign_marker_color(row):
    if row['class'] == 1:
        return 'green'
    elif row['class'] == 0:
        return 'red'

# Apply the function to create the marker_color column
spacex_df['marker_color'] = spacex_df.apply(assign_marker_color, axis=1)



m.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    marker_color = record['marker_color']
    marker_icon = folium.Icon(color='white', icon_color=marker_color)
    marker = folium.Marker(coordinate, icon=marker_icon)
    marker_cluster.add_child(marker)


# Iterate over each launch site
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    launch_site_name = row['Launch Site']

    # Add a Circle object for the launch site with a popup label
    circle = folium.Circle(coordinate, radius=50, color='#d35400', fill=True).add_child(folium.Popup(launch_site_name))
    m.add_child(circle)

    # Add a Marker object for the launch site with a popup label
    marker = folium.Marker(
        coordinate,
        icon=folium.DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>%s</b></div>' % launch_site_name,
        )
    )
    marker.add_child(folium.Popup(launch_site_name))
    m.add_child(marker)

# Display the map
m

In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
# Define a function to create a marker with distance
def create_marker_with_distance(coordinate, distance, name):
    marker = folium.Marker(
        coordinate,
        icon=folium.DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f"<b>{name}</b><br>{distance:.2f} KM"
        )
    )
    return marker

# Find the closest city
city_lat = 28.40167
city_lon = -80.60394
closest_city_coordinate = [city_lat, city_lon]

# Calculating distance
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# Create a marker for the closest city
city_marker = create_marker_with_distance(closest_city_coordinate, distance_city, "Closest City")

# Draw a line between the city marker and the launch site
lines = folium.PolyLine([closest_city_coordinate, (launch_site_lat, launch_site_lon)], weight=1)

# Add the marker and line to the map
m.add_child(city_marker)
m.add_child(lines)

# Display the map
m